In [8]:
import re
from nltk.stem import LancasterStemmer,WordNetLemmatizer
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from gensim.models import *

In [9]:
# Word And Sentence Tokenmization & Stop Removal 
def SentenceTokenization(text):
    sentences = re.compile('[.!?] ').split(text)
    return sentences

def WordTokenization(sentence):
    tokens = re.findall("[\w']+", sentence)
    return tokens

def StopWordRemoval(allwords):
    stopwords = []
    with open("stopwords.txt", 'r') as f:
        stopwords = f.read()
    stopwords = stopwords.split("\n")
    
    usefull_words = [word for word in allwords if word not in stopwords]
    return usefull_words

# Stemming And Lemmatizing
def StemmingAndLemmatizing(usefull_words):
    ls = LancasterStemmer()
    wl = WordNetLemmatizer()
    l=[]
    with open('All_Languages.txt' ,'r+',encoding='utf-8') as f:
        l = f.readlines()

    new_l  =[]
    for i in l:
        new_l.append(i[:-1])
    languages = new_l

    keyword_extract = []
    for i in range(len(usefull_words)):
        for j in range(len(usefull_words[i])):
            if usefull_words[i][j].lower() not in languages:
                x = ls.stem(usefull_words[i][j])
                x = wl.lemmatize(x)
                usefull_words[i][j] = x
            else:
                keyword_extract.append(usefull_words[i][j])
    return usefull_words,keyword_extract

def nlp(text):
    sentences = SentenceTokenization(text)

    words = [] 
    for sentence in sentences:
        words_in_a_sent = WordTokenization(sentence)
        words.append(words_in_a_sent)

    usefull_words = []
    for words_in_a_sentence in words:
        usefull_words_in_sent = StopWordRemoval(words_in_a_sentence)
        usefull_words.append(usefull_words_in_sent)

    usefull_words,keywords = StemmingAndLemmatizing(usefull_words)
    return usefull_words,keywords

In [10]:
training_data = pd.read_csv("DataSet3.csv")

In [11]:
training_data

,X,y
0,3-frcr part 2b preparation course (latest edit...,2b
1,cissp practice tests - all domains - 2020 - 42...,420
2,rhythm #1: intro - turn 8th note to 16th note ...,8th
3,sap abap programming for beginners - online tr...,abap
4,debugging sap abap code for non programmers,abap
...,...,...
3319,introduction to zombie,zombie
3320,basics of zombie,zombie
3321,getting started with zombie,zombie
3322,roadmap for zombie,zombie


In [12]:
training_data = training_data.dropna()

In [13]:
X_train,y_train  = training_data.iloc[:,0].values,training_data.iloc[:,1].values

In [14]:
sent = "Courses on node js not react"
nlp(sent)[1]

['node', 'js', 'not', 'react']

In [15]:
for i in range(len(X_train)):
    X_train[i],keyword = nlp(X_train[i])
    X_train[i] = " ".join(X_train[i][0])
#     print(i+2,keyword)

In [16]:
X_train

array(['3 frcr part 2b prep cours latest edit 2020',
       'cissp pract test domain 2020 420 q part 2',
       'rhythm 1 intro turn 8th not 16th not accompany', ...,
       'get start zombie', 'roadmap zombie', 'want learn zombie'],
      dtype=object)

In [17]:
embeddings_index={}
with open("glove.6B.50d.txt",encoding='utf-8') as f:
    for line in f:
        value = line.split(" ")
        word = value[0]
        coef = np.asarray(value[1:],dtype = 'float32')
        embeddings_index[word] = list(coef)

In [18]:
maxLen = 0
for i in X_train:
    maxLen = max(len(i),maxLen)
print(maxLen)

57


In [19]:
def embedding_output(X):
    emb_dim = 50
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((emb_dim,))
    return embedding_out

embedding_matrix_train = embedding_output(X_train)

In [20]:
embedding_matrix_train.shape

(3324, 57, 50)

In [21]:
with open('All_Languages.txt' ,'r+',encoding='utf-8') as f:
    l = f.readlines()
new_l  =[]
for i in l:
    new_l.append(i[:-1])
l = new_l

d = dict()
rd= dict()
for i in range(len(l)):
    d[l[i]]=i
    rd[i]= l[i]

In [22]:
y_train_num = []
for i in range(len(y_train)):
    y_train_num.append(d[y_train[i]])

In [23]:
y_train_num = to_categorical(np.asarray(y_train_num))

In [24]:
y_train_num.shape

(3324, 672)

In [27]:
model = Sequential()
model.add(LSTM(units = 50,input_shape = (maxLen,50),return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(units=50))
model.add(Dropout(0.5))
model.add(Dense(672))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 57, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 57, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 672)               34272     
_________________________________________________________________
activation_1 (Activation)    (None, 672)               0         
Total params: 74,672
Trainable params: 74,672
Non-trainable params: 0
__________________________________________________

In [28]:
checkpoint = ModelCheckpoint("best_model.h5",monitor = 'val_loss',verbose=False,save_best_only= True)
earlyStop = EarlyStopping(monitor='val_loss')
history = model.fit(embedding_matrix_train,y_train_num,epochs=100,batch_size=64,shuffle=True,validation_split=0.1)

Epoch 1/100
47/47 [==============================] - 3s 63ms/step - loss: 6.1135 - accuracy: 0.0167 - val_loss: 6.8269 - val_accuracy: 0.0000e+00
Epoch 2/100
47/47 [==============================] - 2s 43ms/step - loss: 5.2117 - accuracy: 0.0261 - val_loss: 7.3820 - val_accuracy: 0.0000e+00
Epoch 3/100
47/47 [==============================] - 2s 41ms/step - loss: 5.1368 - accuracy: 0.0221 - val_loss: 7.3265 - val_accuracy: 0.0000e+00
Epoch 4/100
47/47 [==============================] - 2s 40ms/step - loss: 5.1011 - accuracy: 0.0358 - val_loss: 7.2175 - val_accuracy: 0.0000e+00
Epoch 5/100
47/47 [==============================] - 2s 39ms/step - loss: 5.1092 - accuracy: 0.0271 - val_loss: 7.1303 - val_accuracy: 0.0000e+00
Epoch 6/100
47/47 [==============================] - 2s 39ms/step - loss: 5.0869 - accuracy: 0.0274 - val_loss: 7.0715 - val_accuracy: 0.0000e+00
Epoch 7/100
47/47 [==============================] - 2s 40ms/step - loss: 5.0771 - accuracy: 0.0281 - val_loss: 7.0517 - val

In [29]:
model.load_weights("best_model.h5")

ValueError: Shapes (50, 672) and (50, 667) are incompatible

In [86]:
y_test = model.predict(embedding_matrix_train)

In [87]:
for i in range(140):
    print(X_train[i])
    print(rd[np.argmax(y_test[i])])

3 frcr part 2b prep cours latest edit 2020
scratch
cissp pract test domain 2020 420 q part 2
scratch
rhythm 1 intro turn 8th not 16th not accompany
scratch
sap abap program begin onlin train
scratch
debug sap abap cod non program
scratch
sap abap enh mod sap standard
scratch
abc cisco sdwan viptel part 1
scratch
acs exam day ory ndeb equ
scratch
mobl gam develop flash cs6 actionscript 3 0
scratch
actionscript 3 begin adv mobl gam
scratch
ada access m word docu
scratch
introduc algae
scratch
ibm spss amos found cours sem scratch adv
scratch
psychomet u spss amos
scratch
confirm factor analys spss amos
scratch
complet android n develop cours
scratch
complet android oreo develop cours build 23 ap
scratch
android java masterclass becom ap develop
scratch
android ap develop
scratch
adv ap develop android
scratch
build first android ap project cent cours
scratch
angular complet guid 2020 edit
scratch
complet angular cours begin adv
scratch
angular crash cours busy develop
scratch
front end j

In [48]:
embedding_matrix_train.shape

(7870, 236, 50)